<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import findspark
findspark.init()

In [2]:
import os
import time
from PIL import Image
from pyspark import SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.image import ImageSchema

from pyspark.sql.functions import udf
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema

In [3]:
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/jpaul/.conda/envs/OP_8/python.exe'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2,databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 pyspark-shell'

In [4]:
S3 = True
path = 'C:/Users/jpaul/Projet_8/fruits-360/Training/'
bucket = 'op-projet8-julienpaulet2'
categ = os.listdir(path)

In [5]:
def load_aws_key():
    '''charge ID/KEY de AWS dans variables d'environnement si définies sinon dans le fichier keys.txt'''
    with open('Key.txt','r') as f:
        msg = f.read()
    ID = str(msg).split('\n')[0]
    KEY = msg.split('\n')[1]
    os.environ["AWS_ACCESS_KEY_ID"]=ID
    os.environ["AWS_SECRET_ACCESS_KEY"]=KEY
    return ID, KEY

def init_spark_session(S3=False, bucket = '', path_local=''):
    '''Initie une session SPARK
    :s3 : booléen : True = Stockage S3, False = stockage local
    :return SparkContext'''

    if S3 is True:
        path_img = "s3a://"+bucket+"/AppleBraeburn"
        #spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
        #spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3-eu-west-3.amazonaws.com")
        #spark._jsc.hadoopConfiguration().set("fs.s3a.aws.access.key", AWS_ACCESS_KEY_ID)
        #spark._jsc.hadoopConfiguration().set("fs.s3a.aws.secret.key", AWS_SECRET_ACCESS_KEY)
        #spark._jsc.setSystemProperty("com.amazonaws.services.s3.enableV4", "true")
        
        #configuration des identifiants Amazon
        #sc._conf.setAll([
        #('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem'), 
        #('fs.s3a.access.key', AWS_ACCESS_KEY_ID),
        #('fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)])
    else:
        path_img = path_local

    # Instantiation d'une SparkSession et d'un Spark Context
    spark = SparkSession.builder.master('local[*]').appName('P8').getOrCreate()
    sc = spark.sparkContext

    return sc, spark, path_img

if S3 is True:
    load_aws_key()

sc, spark, path = init_spark_session(S3=S3,
                                     bucket=bucket,
                                     path_local=path)

In [6]:
def rename_categ(path, categ):
    """Function that transform the path, removing spaces and point"""
    try:
        for cat in categ:
            if ' ' in cat:
                os.rename(os.path.join(path, cat), os.path.join(path, cat.replace(' ', '')))
            elif '.' in cat:
                os.rename(os.path.join(path, cat), os.path.join(path, cat.replace('.', '')))
    except:
        pass
    cat = os.listdir(path)
    return cat

def parse_category(path):
    '''Renvoie la catégorie d'une image à partir de son chemin'''
    if len(path) > 0:
        #catégorie de l'image
        return path.split('/')[-2]
    else:
        return ''
    
def load_data(path_img):
    '''Chargement des dataframes: 
    Prend en entrée le répertoire qui contient les sous répertoires contenant les images
    Renvoie en sortie un spark dataframe contenant les images et 
    un spark dataframe contenant les noms des fruits associés'''
    #compteur
    start = time.time()
    #chargement dataframe des images
    
    df_img = spark.read.format("image").load([path + cat for cat in categ]) # ne fonctionne pas si il y a des espaces dans le chemin
    #df_img =  ImageSchema.readImages(path_img, dropImageFailures = True)
    print('chargement effectué')
    #récupération chemin à partir des images
    df_img = df_img.withColumn("path", input_file_name())
    #catégorisation des images
    udf_categorie = udf(parse_category, StringType())
    df_img = df_img.withColumn('categorie', udf_categorie('path'))
    print('Temps de chargement des images : {} secondes'.format(time.strftime('%S', time.gmtime(time.time()-start))))
    
    return df_img

def preprocess_data(dataframe):
    '''Renvoie le résultat de l'avant dernière couche de chaque image du dataframe via transform du ResNet50
    return un df contenant des vecteurs de dimension 1x2048 '''
    
    from sparkdl import DeepImageFeaturizer
    # DeepImageFeaturizer Applies the model specified by its popular name, 
    # with its prediction layer(s) chopped off
    featurizer = DeepImageFeaturizer(inputCol="image", outputCol="image_preprocessed", modelName="ResNet50")
    output = featurizer.transform(dataframe).select(['path', 'categorie', 'image_preprocessed'])
    del featurizer
    return output

def write_dataframe_parquet(dataframe, path_parquet):
    '''Enregistrement du spark dataframe au format parquet au chemin specifie'''
    try:
        start = time.time()
        dataframe.write.format("parquet").mode('overwrite').save(path_parquet)
        print('Enregistrement effectué.')

        print('Temps de sauvegarde : {} secondes'.format(time.strftime('%S', time.gmtime(time.time()-start))))
    except:
        print('L\'Enregistrement a échoué.')
    
    return True

def load_dataframe_parquet(path):
    '''chargement du dataframe : entree parquet / sortie dataframe'''
    return spark.read.format('parquet').load('path')

In [7]:
# categ = rename_categ(path, categ)

In [8]:
path

's3a://op-projet8-julienpaulet2/AppleBraeburn'

In [9]:
sc

<SparkContext master=local[*] appName=P8>

In [10]:
#chargement du dataframe contenant les images, leur chemin et leur catégorie
spark_df = load_data(path)

Py4JJavaError: An error occurred while calling o35.load.
: com.amazonaws.services.s3.model.AmazonS3Exception: Status Code: 400, AWS Service: Amazon S3, AWS Request ID: B6542DCCD167B46C, AWS Error Code: null, AWS Error Message: Bad Request, S3 Extended Request ID: mxqozSV2gnb2Q+WA+duaa/MJ5NXq94BB9NBkWz3rlvsMUUJg34q+vbh1JCWTghVjdFZiNSmlayg=
	at com.amazonaws.http.AmazonHttpClient.handleErrorResponse(AmazonHttpClient.java:798)
	at com.amazonaws.http.AmazonHttpClient.executeHelper(AmazonHttpClient.java:421)
	at com.amazonaws.http.AmazonHttpClient.execute(AmazonHttpClient.java:232)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3528)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.

In [ ]:
spark_df.show()

In [ ]:
import numpy as np
a = spark_df.first()
b = np.array(a.asDict()['image']['data']).reshape(100,100,3)[:,:,::-1]
print(b.shape)
Image.fromarray(b, 'RGB')

In [ ]:
# réduction de dimension et détermination des features : avant dernière couche du réseau de neurone ResNet50
spark_df_preprocessed = preprocess_data(spark_df)

In [ ]:
#affichage du dataframe contenant le chemin vers les images, leur catégorie et le vecteur en sortie du Resnet50
spark_df_preprocessed.show()